In [ ]:
%pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate ftfy spacy

In [ ]:
model_id = "stabilityai/stable-diffusion-2-base"
# model_id = "stabilityai/stable-diffusion-2"

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler

scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

In [ ]:
import mediapy as media
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline

device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    torch_dtype=torch.float16,
    revision="fp16",
    )
pipe = pipe.to(device)

if model_id.endswith('-base'):
  image_length = 512
else:
  image_length = 768



In [ ]:
prompt = "a photo of an astronaut riding a horse on mars"
num_images = 1

prompts = [ prompt ] * num_images
with autocast("cuda"):
    images = pipe(
        prompts,
        guidance_scale=7.5,
        num_inference_steps=50,
        height=image_length,
        width=image_length,
        ).images
    
media.show_images(images)
images[0].save("output.jpg")